https://medium.com/the-modern-scientist/graph-neural-networks-series-part-3-node-embedding-36613cc967d5

In [1]:
# import os.path as osp
# from typing import Callable, List, Optional

# import numpy as np
# import torch

# from torch_geometric.data import InMemoryDataset
# from torch_geometric.io import fs, read_planetoid_data


# class Planetoid1(InMemoryDataset):
#     r"""The citation network datasets :obj:`"Cora"`, :obj:`"CiteSeer"` and
#     :obj:`"PubMed"` from the `"Revisiting Semi-Supervised Learning with Graph
#     Embeddings" <https://arxiv.org/abs/1603.08861>`_ paper.
#     Nodes represent documents and edges represent citation links.
#     Training, validation and test splits are given by binary masks.

#     Args:
#         root (str): Root directory where the dataset should be saved.
#         name (str): The name of the dataset (:obj:`"Cora"`, :obj:`"CiteSeer"`,
#             :obj:`"PubMed"`).
#         split (str, optional): The type of dataset split (:obj:`"public"`,
#             :obj:`"full"`, :obj:`"geom-gcn"`, :obj:`"random"`).
#             If set to :obj:`"public"`, the split will be the public fixed split
#             from the `"Revisiting Semi-Supervised Learning with Graph
#             Embeddings" <https://arxiv.org/abs/1603.08861>`_ paper.
#             If set to :obj:`"full"`, all nodes except those in the validation
#             and test sets will be used for training (as in the
#             `"FastGCN: Fast Learning with Graph Convolutional Networks via
#             Importance Sampling" <https://arxiv.org/abs/1801.10247>`_ paper).
#             If set to :obj:`"geom-gcn"`, the 10 public fixed splits from the
#             `"Geom-GCN: Geometric Graph Convolutional Networks"
#             <https://openreview.net/forum?id=S1e2agrFvS>`_ paper are given.
#             If set to :obj:`"random"`, train, validation, and test sets will be
#             randomly generated, according to :obj:`num_train_per_class`,
#             :obj:`num_val` and :obj:`num_test`. (default: :obj:`"public"`)
#         num_train_per_class (int, optional): The number of training samples
#             per class in case of :obj:`"random"` split. (default: :obj:`20`)
#         num_val (int, optional): The number of validation samples in case of
#             :obj:`"random"` split. (default: :obj:`500`)
#         num_test (int, optional): The number of test samples in case of
#             :obj:`"random"` split. (default: :obj:`1000`)
#         transform (callable, optional): A function/transform that takes in an
#             :obj:`torch_geometric.data.Data` object and returns a transformed
#             version. The data object will be transformed before every access.
#             (default: :obj:`None`)
#         pre_transform (callable, optional): A function/transform that takes in
#             an :obj:`torch_geometric.data.Data` object and returns a
#             transformed version. The data object will be transformed before
#             being saved to disk. (default: :obj:`None`)
#         force_reload (bool, optional): Whether to re-process the dataset.
#             (default: :obj:`False`)

#     **STATS:**

#     .. list-table::
#         :widths: 10 10 10 10 10
#         :header-rows: 1

#         * - Name
#           - #nodes
#           - #edges
#           - #features
#           - #classes
#         * - Cora
#           - 2,708
#           - 10,556
#           - 1,433
#           - 7
#         * - CiteSeer
#           - 3,327
#           - 9,104
#           - 3,703
#           - 6
#         * - PubMed
#           - 19,717
#           - 88,648
#           - 500
#           - 3
#     """
#     url = 'https://gitee.com/jiajiewu/planetoid/raw/master/data'
#     geom_gcn_url = ('https://raw.githubusercontent.com/graphdml-uiuc-jlu/'
#                     'geom-gcn/master')

#     def __init__(
#         self,
#         root: str,
#         name: str,
#         split: str = "public",
#         num_train_per_class: int = 20,
#         num_val: int = 500,
#         num_test: int = 1000,
#         transform: Optional[Callable] = None,
#         pre_transform: Optional[Callable] = None,
#         force_reload: bool = False,
#     ) -> None:
#         self.name = name

#         self.split = split.lower()
#         assert self.split in ['public', 'full', 'geom-gcn', 'random']

#         super().__init__(root, transform, pre_transform,
#                          force_reload=force_reload)
#         self.load(self.processed_paths[0])

#         if split == 'full':
#             data = self.get(0)
#             data.train_mask.fill_(True)
#             data.train_mask[data.val_mask | data.test_mask] = False
#             self.data, self.slices = self.collate([data])

#         elif split == 'random':
#             data = self.get(0)
#             data.train_mask.fill_(False)
#             for c in range(self.num_classes):
#                 idx = (data.y == c).nonzero(as_tuple=False).view(-1)
#                 idx = idx[torch.randperm(idx.size(0))[:num_train_per_class]]
#                 data.train_mask[idx] = True

#             remaining = (~data.train_mask).nonzero(as_tuple=False).view(-1)
#             remaining = remaining[torch.randperm(remaining.size(0))]

#             data.val_mask.fill_(False)
#             data.val_mask[remaining[:num_val]] = True

#             data.test_mask.fill_(False)
#             data.test_mask[remaining[num_val:num_val + num_test]] = True

#             self.data, self.slices = self.collate([data])

#     @property
#     def raw_dir(self) -> str:
#         if self.split == 'geom-gcn':
#             return osp.join(self.root, self.name, 'geom-gcn', 'raw')
#         return osp.join(self.root, self.name, 'raw')

#     @property
#     def processed_dir(self) -> str:
#         if self.split == 'geom-gcn':
#             return osp.join(self.root, self.name, 'geom-gcn', 'processed')
#         return osp.join(self.root, self.name, 'processed')

#     @property
#     def raw_file_names(self) -> List[str]:
#         names = ['x', 'tx', 'allx', 'y', 'ty', 'ally', 'graph', 'test.index']
#         return [f'ind.{self.name.lower()}.{name}' for name in names]

#     @property
#     def processed_file_names(self) -> str:
#         return 'data.pt'

#     def download(self) -> None:
#         for name in self.raw_file_names:
#             fs.cp(f'{self.url}/{name}', self.raw_dir)
#         if self.split == 'geom-gcn':
#             for i in range(10):
#                 url = f'{self.geom_gcn_url}/splits/{self.name.lower()}'
#                 fs.cp(f'{url}_split_0.6_0.2_{i}.npz', self.raw_dir)

#     def process(self) -> None:
#         data = read_planetoid_data(self.raw_dir, self.name)

#         if self.split == 'geom-gcn':
#             train_masks, val_masks, test_masks = [], [], []
#             for i in range(10):
#                 name = f'{self.name.lower()}_split_0.6_0.2_{i}.npz'
#                 splits = np.load(osp.join(self.raw_dir, name))
#                 train_masks.append(torch.from_numpy(splits['train_mask']))
#                 val_masks.append(torch.from_numpy(splits['val_mask']))
#                 test_masks.append(torch.from_numpy(splits['test_mask']))
#             data.train_mask = torch.stack(train_masks, dim=1)
#             data.val_mask = torch.stack(val_masks, dim=1)
#             data.test_mask = torch.stack(test_masks, dim=1)

#         data = data if self.pre_transform is None else self.pre_transform(data)
#         self.save([data], self.processed_paths[0])

#     def __repr__(self) -> str:
#         return f'{self.name}()'

# ## 如果数据下载不下来，可以尝试这么做：获取planetoid的pyg源代码，改一下里面的库的url，改一个国内的库。
# dataset = Planetoid1(name='Cora', root="data/")

In [2]:
# import torch
# from torch_geometric.data import Data

# # Define the adjacency matrix (edge connections)
# edge_index = torch.tensor([[0, 1, 1, 2, 3, 3, 4, 4], [1, 0, 2, 1, 2, 4, 3, 5]], dtype=torch.long)

# # Define the node features
# node_features = torch.tensor([[100, 5.0, 20], [200, 4.2, 15], [150, 4.8, 18], [180, 4.6, 25], [120, 4.9, 12], [250, 4.1, 30]], dtype=torch.float)

# # Create the PyTorch Geometric Data object
# data = Data(x=node_features, edge_index=edge_index)

# # Print the data object
# print(data)

In [3]:
import pandas as pd
import numpy as np
import time

In [4]:
df_ori = pd.read_csv("../torch/df_ori.csv")

In [5]:
df_ori

,investor_id,company_id
0,393,393
1,2787,2790
2,666,799
3,1084,1866
4,1084,1983
...,...,...
2050,768,2404
2051,482,2418
2052,1515,2719
2053,622,623


In [6]:
df_feaVal = pd.read_csv("../torch/test.csv")
df_feaVal = df_feaVal.fillna(-1) ## 似乎把这里填满，会有更好效果。而且本来，神经网络就不支持空值啊。
fea_list = df_feaVal.iloc[:, 6:-1].columns 

In [19]:
## 下面的代码来源于最上面提到的页面。
## 写得非常翔实。代码几乎直接就能跑。
import os.path as osp

import torch
from sklearn.metrics import roc_auc_score

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling
from torch_geometric.data import Data

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device),
    T.RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True,
                      add_negative_train_samples=False),
])
# path = osp.join(osp.dirname(osp.realpath(__file__)), '..', 'data', 'Planetoid')
# dataset = Planetoid(name='Cora', transform=transform, root="data/")

ori_dataset = Data(
    x = torch.tensor(
        df_feaVal.iloc[:, 6:-1].to_numpy(), 
        dtype=torch.float32
    ),
    num_nodes = df_feaVal.shape[0],
    edge_index=torch.tensor(
        df_ori.T.to_numpy(), 
        dtype = torch.int64
    ), 
    num_features = len(fea_list)
)

# After applying the `RandomLinkSplit` transform, the data is transformed from
# a data object to a list of tuples (train_data, val_data, test_data), with
# each element representing the corresponding split.

train_data, val_data, test_data = transform(ori_dataset) #[0]


class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()


model = Net(len(fea_list), 128, 64).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()


def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    # We perform a new round of negative sampling for every training epoch:
    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

    # Concat positive and negative edge indices.
    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    # Label for positive edges: 1, for negative edges: 0.
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)

    # Note: The model is trained in a supervised manner using the given
    # `edge_label_index` and `edge_label` targets.
    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()
    return loss


@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())

# Train/Test Loop
best_val_auc = final_test_auc = 0
for epoch in range(1, 5):
    t1 = time.time()
    loss = train()
    print(loss)
    val_auc = test(val_data)
    test_auc = test(test_data)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        final_test_auc = test_auc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
          f'Test: {test_auc:.4f}')
    t2 = time.time()
    print("use time in minutes:", (t2-t1)/60, "\n")

print(f'Final Test: {final_test_auc:.4f}')

# z = model.encode(test_data.x, test_data.edge_index)
# final_edge_index = model.decode_all(z)

df_new = pd.concat(
    [
        df_feaVal[["company_id"]].reset_index(drop=True), 
        pd.DataFrame(
            model.encode(ori_dataset.x, ori_dataset.edge_index).detach().numpy(), 
            columns=[f"col_{i}" for i in range(64)]
        ).reset_index(drop=True)
    ], 
    axis=1
)

tensor(0.6903, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
Epoch: 001, Loss: 0.6903, Val: 0.8431, Test: 0.8414
use time in minutes: 0.00028607845306396487 

tensor(0.6443, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
Epoch: 002, Loss: 0.6443, Val: 0.8448, Test: 0.8399
use time in minutes: 0.00027417739232381185 

tensor(0.6339, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
Epoch: 003, Loss: 0.6339, Val: 0.8473, Test: 0.8395
use time in minutes: 0.00024131536483764648 

tensor(0.6316, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
Epoch: 004, Loss: 0.6316, Val: 0.8481, Test: 0.8400
use time in minutes: 0.0002474188804626465 

Final Test: 0.8400
